## S2 To R1 Attractor + R1 Obstacle Avoidance

In [ ]:
using PBDS, StaticArrays, LinearAlgebra, GeometryBasics, Rotations, BenchmarkTools

## Setup

### Point Attractor

In [ ]:
PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::DistanceFromPoint{S2,R1}) = SA[norm(xme - task_map.position_center)]
PBDS.domain_coord_rep(::DistanceFromPoint{S2,R1}) = EmbRep()

PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = 0*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{S2,R1}}, CN::Chart{1,R1}) = 
    default_weight_metric(xn, vn, task, CN)

### Sphere Damping

In [ ]:
PBDS.default_coord_rep(::Damping{<:Identity{S2,S2}}) = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:Identity{S2,S2}}) =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:Identity{S2,S2}}) = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:Identity{S2,S2}}) = -4*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:Identity{S2,S2}}) =
    default_weight_metric(xne, vne, task)
PBDS.home_task_chart(task::Damping{<:Identity{S2,S2}}) = Chart{SterProjSouth,S2}()

### Sphere Avoidance

In [ ]:
PBDS.domain_coord_rep(::DistanceFromSphereSurface{S2,R1}) = EmbRep()
PBDS.task_map_emb(::EmbRep, ::EmbRep, xme, task_map::DistanceFromSphereSurface{S2,R1}) =
    SA[abs(norm(xme - task_map.center) - task_map.radius)]

function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{S2,R1}}, CN::Chart{1,R1})
    ψx = exp(1.e0 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end
PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{S2,R1}}, CN::Chart{1,R1}) = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{S2,R1}}, CN::Chart{1,R1}) = 0*vn
function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{S2,R1}}, CN::Chart{1,R1})
    offset_distance = 5.
    λ = (xn[1] < offset_distance && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

In [ ]:
M = S2
CM = Chart{SterProjNorth,S2}()
tasks, CNs = TaskList(), ChartList()

N = R1
CN = Chart{1,N}()
xm_goal = SA[1., -1., -1.]
xm_goal = xm_goal ./ norm(xm_goal)
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, CN)

N = S2
CN = Chart{SterProjSouth,S2}()
push!(tasks, Damping(Identity{M,N,Float64}()))
push!(CNs, CN)

N = R1
CN = Chart{1,N}()
obs_centers = []
obs_radii = []

push!(obs_centers, SA[1., 1.2, 0.8])
push!(obs_radii, 0.4)

push!(obs_centers, SA[-0.2, 0.2, 1.])
push!(obs_radii, 0.3)

push!(obs_centers, SA[0.2, -0.2, -1.])
push!(obs_radii, 0.3)

push!(obs_centers, SA[-1.2, 1., -0.8])
push!(obs_radii, 0.4)

push!(obs_centers, SA[-1.2, -0.9, -0.1])
push!(obs_radii, 0.3)

for i in 1:length(obs_centers)
    obs_centers[i] = (obs_centers[i] / norm(obs_centers[i])) * (1. - obs_radii[i]/2)
    push!(tasks, CollisionAvoidance(DistanceFromSphereSurface{M,N}(obs_centers[i], obs_radii[i])))
    push!(CNs, CN)
end

In [ ]:
# Initial state
xme = SA[-1., 1., 1.]
xme = xme ./ norm(xme)
vme = SA[2., 1., 1.]

robot_coord_rep = EmbRep()
σxddot, = multiple_task_acceleration(xme, vme, tasks, CM, CNs, robot_coord_rep)

## Single Trajectory

In [ ]:
using Plots, Makie, Observables, ProgressMeter, ColorSchemes

In [ ]:
Time = 40
dt = 0.01

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjNorth,S2}()
traj_north = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) = Chart{SterProjSouth,S2}()
traj_south = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)

PBDS.choose_chart_emb(::EmbRep, pe, ::Chart{<:SterProj,S2}) =
    (pe[3] < 0)[1] ? Chart{SterProjSouth,S2}() : Chart{SterProjNorth,S2}()
traj_switching = propagate_tasks(xme, vme, tasks, CM, CNs, Time, dt, robot_coord_rep, log_tasks = true)
traj_switching.xm[end]

In [ ]:
Nplot = length(traj_switching.xm)
if !isdefined(Main, :no_plots)
    Plots.plot(getindex.(traj_switching.xm,1)[1:Nplot])
    Plots.plot!(getindex.(traj_switching.xm,2)[1:Nplot])
    Plots.plot!(getindex.(traj_switching.xm,3)[1:Nplot])
end

In [ ]:
ax_size, plot_size = 1, 800
scene = Scene(resolution = (plot_size, plot_size))
mesh!(Sphere(Point3(zeros(3)), 1.), color = RGBA(1.,1.,1.,0.7), transparency = true)
for i in 1:length(obs_centers)
    mesh!(Sphere(Point3(obs_centers[i]...), obs_radii[i]), color = RGBA(0.,1.,0.,0.4), transparency = true)
end
Makie.scatter!(scene, [xme[1]], [xme[2]], [xme[3]], markersize = ax_size/20, color = :blue)
δ = 0.99
Makie.scatter!(scene, [xm_goal[1]*δ], [xm_goal[2]*δ], [xm_goal[3]*δ], markersize = ax_size/20, color = :green)

linewidth = 1.5
Makie.lines!(scene, getindex.(traj_switching.xm,1), getindex.(traj_switching.xm,2), getindex.(traj_switching.xm,3), color = :purple, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_south.xm,1), getindex.(traj_south.xm,2), getindex.(traj_south.xm,3), color = :red, linewidth = linewidth)
Makie.lines!(scene, getindex.(traj_north.xm,1), getindex.(traj_north.xm,2), getindex.(traj_north.xm,3), color = :blue, linewidth = linewidth)

Makie.xlims!(scene, (-ax_size, ax_size))
Makie.ylims!(scene, (-ax_size, ax_size))
Makie.zlims!(scene, (-ax_size, ax_size))
axis = scene[Axis]
axis.showaxis = false
rotate_cam!(scene, 0.4, 0., 0.)
isdefined(Main, :no_plots) || display(scene)

## Multiple Trajectories

In [ ]:
Time = 18
dt = 0.05

θ = collect(0:π/25:2π)
ntraj = length(θ) - 1

vm_gen = []
for i in 1:ntraj
    R = AngleAxis(θ[i], xme...)
    push!(vm_gen, R*vme)
end

trajs = Array{Any}(undef, ntraj)
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(xme, vm_gen[i], tasks, CM, CNs, Time, dt, robot_coord_rep)
    isdefined(Main, :Test) || next!(p)
end

In [ ]:
iobs = Observable(1)

ax_size, plot_size = 1, 800
limits = FRect3D((-ax_size, -ax_size, -ax_size), (2*ax_size, 2*ax_size, 2*ax_size))
scene = Scene(resolution = (plot_size, plot_size))

mesh!(Sphere(Point3(zeros(3)), 1.), color = RGBA(1.,1.,1.,0.0))
for i in 1:length(obs_centers)
    color = RGBA(0.95,0.6,0.3,1.) # Orange
    mesh!(Sphere(Point3(obs_centers[i]...), obs_radii[i]), shininess = 8.0f0, specular = Vec3f0(0.15); color)
end
for i = 1:ntraj
    color = get(ColorSchemes.vikO, θ[i]/(2π))
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), getindex.(trajs[i].xm,3); color, linewidth = 2)
    Makie.scatter!(scene, lift(j -> [trajs[i].xm[j][1]], iobs), lift(j -> [trajs[i].xm[j][2]], iobs), lift(j -> [trajs[i].xm[j][3]], iobs), markersize = ax_size/20; color)
end
Makie.scatter!(scene, [xm_goal[1]*δ], [xm_goal[2]*δ], [xm_goal[3]*δ], markersize = ax_size/15, color = :green)

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
rotate_cam!(scene, 0.4, 0., 0.)
isdefined(Main, :no_plots) || display(scene)

In [ ]:
function record_scene(scene, filename, iobs, N, framerate=60)
    p = Progress(N)
    record(scene, filename, 1:N) do i
        iobs[] = i
        rotate_cam!(scene, 0.015, 0., 0.)
        isdefined(Main, :Test) || next!(p)
    end
    isdefined(Main, :Test) || display("text/html", html_video(filename))
end

filename = "S2_To_R1Attractor_S2Damping_R1ObstacleAvoidance.mp4"
isdefined(Main, :no_plots) || record_scene(scene, filename, iobs, length(trajs[1].xm))

In [ ]:
iobs = Observable(1)
scene = Scene(resolution = (plot_size, plot_size))

mesh!(Sphere(Point3(zeros(3)), 1.), color = RGBA(1.,1.,1.,0.0))
for i in 1:length(obs_centers)
    color = RGBA(0.95,0.6,0.3,1.) # Orange
    mesh!(Sphere(Point3(obs_centers[i]...), obs_radii[i]), shininess = 8.0f0, specular = Vec3f0(0.15); color)
end
N = length(trajs[1].xm)
for i = 1:ntraj
    color = get(ColorSchemes.vikO, θ[i]/(2π))
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), getindex.(trajs[i].xm,3); color, linewidth = 2)
    Makie.scatter!(scene, lift(j -> [trajs[i].xm[mod(j-1,N)+1][1]], iobs), lift(j -> [trajs[i].xm[mod(j-1,N)+1][2]], iobs), lift(j -> [trajs[i].xm[mod(j-1,N)+1][3]], iobs), markersize = ax_size/20; color)
end
Makie.scatter!(scene, [xm_goal[1]*δ], [xm_goal[2]*δ], [xm_goal[3]*δ], markersize = ax_size/15, color = :green)

Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
axis.showaxis = false
rotate_cam!(scene, 0.4, 0., 0.)
isdefined(Main, :no_plots) || display(scene)

In [ ]:
filename = "S2_To_R1Attractor_S2Damping_R1ObstacleAvoidance_repeated.mp4"
isdefined(Main, :no_plots) || record_scene(scene, filename, iobs, 2*length(trajs[1].xm))